## An year is a Leap Year

In [1]:
import calendar 
print(calendar.isleap(1900))

False


# Determining the Number of days in a year

In [2]:
year = 1965
import calendar 
if calendar.isleap(year):
    print(f"The year {year} consists of 366 days") 
else: 
    print(f"The year {year} consists of 365 days")

The year 1965 consists of 365 days


# Extracting Units of Time from a Date

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/12/20 13:35:21 WARN Utils: Your hostname, user-HP-Pavilion-x360-Convertible-14-dh0xxx resolves to a loopback address: 127.0.1.1; using 192.168.122.1 instead (on interface virbr0)
24/12/20 13:35:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/20 13:35:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [25]:
from pyspark.sql.functions import *

# create sample Dataframe
data = [("2022-03-15", "2022-03-16 12:34:56.789"), 
        ("2022-03-01", "2022-03-16 01:23:45.678")] 
df = spark.createDataFrame(data, ["date_col","timestamp_col"]) 
df.show(truncate=False)

+----------+-----------------------+
|date_col  |timestamp_col          |
+----------+-----------------------+
|2022-03-15|2022-03-16 12:34:56.789|
|2022-03-01|2022-03-16 01:23:45.678|
+----------+-----------------------+



In [5]:
df.select(year("date_col").alias("years"),month("date_col").alias("months"),dayofmonth("date_col")).show()

+-----+------+--------------------+
|years|months|dayofmonth(date_col)|
+-----+------+--------------------+
| 2022|     3|                  15|
| 2022|     3|                   1|
+-----+------+--------------------+



In [6]:
df.select(hour("timestamp_col"),minute("timestamp_col"),second("timestamp_col")).show()

+-------------------+---------------------+---------------------+
|hour(timestamp_col)|minute(timestamp_col)|second(timestamp_col)|
+-------------------+---------------------+---------------------+
|                 12|                   34|                   56|
|                  1|                   23|                   45|
+-------------------+---------------------+---------------------+



# Determining the First and Last Days of a month

In [7]:
df.withColumn("first day", trunc("date_col","month")).show()

+----------+--------------------+----------+
|  date_col|       timestamp_col| first day|
+----------+--------------------+----------+
|2022-03-15|2022-03-16 12:34:...|2022-03-01|
|2022-03-01|2022-03-16 01:23:...|2022-03-01|
+----------+--------------------+----------+



# Last Day of a month

In [ ]:
df.select(last_day(df.timestamp_col).alias("last_day")).show()

# Determining All dates for a Particular Weekday Throughout a Year. 

In [ ]:
df.withColumn("day_of_week",dayofweek('date_col')).show()

In [ ]:
df.withColumn("Quarter", quarter('date_col')).show()

In [ ]:
df.withColumn('last_day',expr('''to_date(date_trunc("quarter", to_date(input_date) + interval 3 months) + interval 1 day '''))

In [32]:
df2=df.withColumn("first_day", trunc("date_col","month"))
df2.show()

+----------+--------------------+----------+
|  date_col|       timestamp_col| first_day|
+----------+--------------------+----------+
|2022-03-15|2022-03-16 12:34:...|2022-03-01|
|2022-03-01|2022-03-16 01:23:...|2022-03-01|
+----------+--------------------+----------+



In [33]:
df2.select("date_col", add_months(df2.first_day,3).alias("last_day")).show()

+----------+----------+
|  date_col|  last_day|
+----------+----------+
|2022-03-15|2022-06-01|
|2022-03-01|2022-06-01|
+----------+----------+



## Listing Quarter Start and End Dates for the Year

### start with the current date

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
from pyspark.sql.functions import current_date, trunc, year, lit, make_date,add_months
data = [[1]] 
df=spark.createDataFrame(data=data).toDF("id")
df.show()
df2=df.withColumn("current_date",current_date())
df3=df.select(year("current_date").alias('year'))
df3.show()
year = df3.collect()[0][0]
print(f"year {year}")


+---+
| id|
+---+
|  1|
+---+

+----+
|year|
+----+
|2024|
+----+

year 2024


### Better Code for generating a date in a year

In [18]:
date_data = [(2024, 1, 1)] 
df4=spark.createDataFrame(data=date_data).toDF("year","month","day")
df4.show()
df5 = df4.select(make_date(df4.year,df4.month,df4.day).alias('datefield')) #.show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2024|    1|  1|
+----+-----+---+



In [14]:
df6=df5.withColumn("end_date",add_months(df5.datefield,3))
df6.show()

+----------+----------+
| datefield|  end_date|
+----------+----------+
|2024-01-01|2024-04-01|
+----------+----------+



In [19]:

df7=df6.withColumnRenamed("datefield","start_date").withColumn('quarter',lit(1))
df7.show()

+----------+----------+-------+
|start_date|  end_date|quarter|
+----------+----------+-------+
|2024-01-01|2024-04-01|      1|
+----------+----------+-------+



In [23]:
from pyspark.sql.functions import date_sub, date_add
df8=df7.withColumn("end_date",date_sub(df7.end_date, 1)) #.show()
df8.show()

+----------+----------+-------+
|start_date|  end_date|quarter|
+----------+----------+-------+
|2024-01-01|2024-03-31|      1|
+----------+----------+-------+



In [34]:
start_date_2Q = df8.select(date_add(df8.end_date, 1)).collect()[0][0]
print(type(start_date_2Q))
end_date_2Q = df8.select(add_months(df8.end_date, 3)).collect()[0][0]
print(end_date_2Q)
data_Q2 = [(start_date_2Q, end_date_2Q, 2)]
col=("start_date", "end_date", "quarter")
df_Q2 = spark.createDataFrame(data=data_Q2, schema=col)
df_Q2.show()
df8.union(df_Q2).show()

<class 'datetime.date'>
2024-06-30
+----------+----------+-------+
|start_date|  end_date|quarter|
+----------+----------+-------+
|2024-04-01|2024-06-30|      2|
+----------+----------+-------+

+----------+----------+-------+
|start_date|  end_date|quarter|
+----------+----------+-------+
|2024-01-01|2024-03-31|      1|
|2024-04-01|2024-06-30|      2|
+----------+----------+-------+



## Given the current date, find which quarter it belongs to:


In [28]:
from pyspark.sql.functions import current_date
data=[[1]];col=['id'] 
df = spark.createDataFrame(data=data, schema=col) 
df2=df.withColumn("current_date",current_date()) #.show()

In [36]:
from pyspark.sql.functions import month
month = df.select(month("current_date").alias("month")).collect()[0][0]
print("month ",month)
print(f"current date is in {month//3} quarter")

month  12
current date is in 4 quarter
